In [1]:
import cv2
import pytesseract
import numpy as np

In [2]:
config_tesseract = "--tessdata-dir tessdata --psm 6"

In [3]:
imagem = cv2.imread('imagens/placa_carro3.jpg')
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
cv2.imshow('image', imagem)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,13))

chapeu_preto = cv2.morphologyEx(imagem, cv2.MORPH_BLACKHAT, kernel)
cv2.imshow('image', chapeu_preto)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
sobel_x = cv2.Sobel(chapeu_preto, ddepth=cv2.CV_32F, dx= 1, dy= 0, ksize=1)
sobel_x = np.absolute(sobel_x)
sobel_x = sobel_x.astype('uint8')
cv2.imshow('image', sobel_x)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
sobel_x = cv2.GaussianBlur(sobel_x, (5,5), 0)
sobel_x = cv2.morphologyEx(sobel_x, cv2.MORPH_CLOSE, kernel)
cv2.imshow('image', sobel_x)
cv2.waitKey(0)
cv2.destroyAllWindows()
valor, limiar = cv2.threshold(sobel_x, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow('image', limiar)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
kernel_quad = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
limiar = cv2.erode(limiar, kernel_quad, iterations=2)
limiar = cv2.dilate(limiar, kernel_quad, iterations=2)
cv2.imshow('image', limiar)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
fechamento = cv2.morphologyEx(imagem, cv2.MORPH_CLOSE, kernel_quad)
valor, mascara = cv2.threshold(fechamento, 0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow('image', mascara)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
limiarizacao = cv2.bitwise_and(limiar, limiar, mask=mascara)
cv2.imshow('image', limiarizacao)
cv2.waitKey(0)
cv2.destroyAllWindows()
limiarizacao = cv2.dilate(limiarizacao, kernel_quad, iterations=2)
cv2.imshow('image', limiarizacao)
cv2.waitKey(0)
cv2.destroyAllWindows()
limiarizacao = cv2.erode(limiarizacao, kernel_quad)
cv2.imshow('image', limiarizacao)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
from skimage.segmentation import clear_border
limiarizacao = clear_border(limiarizacao)
cv2.imshow('image', limiarizacao)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
contornos, hierarquia = cv2.findContours(limiarizacao, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contornos = sorted(contornos, key = cv2.contourArea, reverse = True)[:10]
contornos


[array([[[397, 195]],
 
        [[397, 197]],
 
        [[396, 198]],
 
        [[396, 202]],
 
        [[397, 203]],
 
        [[397, 255]],
 
        [[403, 255]],
 
        [[405, 253]],
 
        [[405, 252]],
 
        [[406, 251]],
 
        [[446, 251]],
 
        [[448, 253]],
 
        [[450, 253]],
 
        [[450, 249]],
 
        [[451, 248]],
 
        [[453, 248]],
 
        [[454, 249]],
 
        [[454, 253]],
 
        [[458, 253]],
 
        [[460, 251]],
 
        [[463, 251]],
 
        [[463, 248]],
 
        [[464, 247]],
 
        [[534, 247]],
 
        [[535, 248]],
 
        [[535, 252]],
 
        [[537, 252]],
 
        [[538, 253]],
 
        [[540, 253]],
 
        [[540, 242]],
 
        [[541, 241]],
 
        [[543, 241]],
 
        [[544, 242]],
 
        [[544, 247]],
 
        [[545, 248]],
 
        [[545, 253]],
 
        [[548, 253]],
 
        [[549, 252]],
 
        [[549, 251]],
 
        [[550, 250]],
 
        [[550, 248]],
 
        [[551, 2

In [12]:
for contorno in contornos:
    x, y, w, h = cv2.boundingRect(contorno)
    proporcao = float(w)/h
    if proporcao >= 3 and proporcao <= 3.5:
        placa = imagem[y:y+h, x:x+w]
        valor, regiao_interesse = cv2.threshold(placa,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        regiao_interesse = clear_border(regiao_interesse)
        cv2.imshow('image', placa)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('image', regiao_interesse)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break

In [13]:
texto= pytesseract.image_to_string(regiao_interesse, lang = 'por', config=config_tesseract)
print(texto)

.BDM3D69



In [14]:
import re

texto_extraido = re.search('\w{3}\d{1}\w{1}\d{2}', texto)

texto_extraido

print(texto_extraido.group(0))

BDM3D69
